## RAG with Anthropic Claude-3-5-sonnet
Author: **Peeyush Sharma**; Feedback: **PSharma3@gmail.com**

This notebook demonstrates use of web search as an agentic tool. We are retrieving the most recent published values of a few major central bank rates (SOFR, SONIA). Specific target sites have been provided to the tool to ensure veracity and authenticity of the information retrieved. The tool is expected to contain its search and response within these target sites. The thought behind this type of tool is to demonstrate that agents can be used to systematically traverse through a range of sites retrieving specific information from each of them and aggregate the final collection in the user requested format. In this case, this can be something like:

* Date: Rate: Value
** 2025-06-12: SOFR: 4.32
** 2025-06-12: SONIA: 3.85
** 2025-06-12: ESTR: 1.25

In [17]:
import os
from anthropic import Anthropic, RateLimitError
from dotenv import load_dotenv
from time import sleep

In [18]:
# A dictionary for reference rate and their official source.
# Agentic tool will be limited to retrieval from these sources only.
# Can potentially add more sources for cross-checks (more LLM calls AND more confidence)
rate_url_dict = {
    "SOFR": ["newyorkfed.org/markets/reference-rates/sofr",
             "fred.stlouisfed.org/series/SOFR"],
    "SONIA": ["bankofengland.co.uk"],
    # "SARON": ["six-group.com/en/market-data/indices/switzerland/saron.html"],
    # "ESTR": ["ecb.europa.eu/stats/financial_markets_and_interest_rates/euro_short-term_rate/html/index.en.html"],
}


In [19]:
def llm_call(prompt: str, tools: str = "", system_prompt: str = "", model="claude-3-5-sonnet-20241022") -> str:
    """
    Calls the model with the given prompt and returns the response.

    Args:
        prompt (str): The user prompt to send to the model.
        tools (str): The tools used by the model. Defaults to basic web search.
        system_prompt (str, optional): The system prompt to send to the model. Defaults to "".
        model (str, optional): The model to use for the call. Defaults to "claude-3-5-sonnet-20241022".

    Returns:
        str: The response from the language model.
    """
    load_dotenv()
    anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
    seconds_sleep = 60

    # client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
    client = Anthropic(api_key=anthropic_api_key)
    messages = [
        {"role": "user", "content": prompt}]
    if tools == '':
        tools = [{
            "type": "web_search_20250305",
            "name": "web_search",
            "max_uses": 5
        }]

    try:
        ret_response = client.messages.create(
            model=model,
            max_tokens=4096,
            system=system_prompt,
            messages=messages,
            tools=tools,
            temperature=0
        )
    except RateLimitError as err:
        print(err)
        print(f'Pausing for {seconds_sleep} seconds before next attempt.')
        sleep(seconds_sleep) #
        ret_response = llm_call(prompt, tools)
    return ret_response

In [20]:
def populate_request(reference_rate: str, urls: list) -> tuple[str, list[dict]]:
    prompt = f"""
            Find the most recent {reference_rate} rate published by Federal Reserve Bank of New York.
                       Extract only the numerical {reference_rate} values and associated date from the web page.
                       Respond in format 'Date: Value'. Example format:
                        2025-06-12: 4.28"""
    web_search_dict = {
            "name": "web_search",
            "type": "web_search_20250305",
            "max_uses": 5,
            "allowed_domains": urls,
            "user_location": {
                "type": "approximate",
                "city": "New York",
                "region": "New York",
                "country": "US",
                "timezone": "America/New_York",
            }
    }
    web_tools = [web_search_dict]
    return prompt, web_tools

for ref_rate, urls in rate_url_dict.items():
    prompt, tools = populate_request(ref_rate, urls)
    response = llm_call(prompt, tools)
    # Extract and print the content
    print("\n----------------------")
    print("Response content:")
    for content_block in response.content:
        if content_block.type == "text":
            print(content_block.text)

    # Optionals
    # Print the full response
    # print("\nFull response:")
    # print(response.model_dump_json(indent=2))


    # Print usage information
    # print("\nUsage statistics:")
    # print(f"Input tokens: {response.usage.input_tokens}")
    # print(f"Output tokens: {response.usage.output_tokens}")
    # if response.usage.server_tool_use:
    #     print(f"Web search requests: {response.usage.server_tool_use.web_search_requests}")


----------------------
Response content:
Let me search for the most recent SOFR rate.
Let me search specifically for the most recent SOFR rate value.
Let me try one more specific search to get the latest rate value.
Based on the search results, I can provide you with the most recent SOFR rate:


2025-06-18: 4.28


This is the latest available rate, which 
was updated on June 20, 2025 at 7:02 AM CDT
. For context, 
the Secured Overnight Financing Rate (SOFR) is a broad measure of the cost of borrowing cash overnight collateralized by Treasury securities
, and 
it is published by the New York Fed each business day at approximately 8:00 a.m. ET
.

----------------------
Response content:
Let me search for the most recent SONIA rate.
Let me search specifically for the most recent SONIA rate value.
Let me try one more specific search to find the current SONIA rate value.
Based on the search results, I can provide the following information about the current SONIA rate:


The SONIA rate for 

## TODO:
- Add recursive prompts to export only the rate, date, and value triplet and remove any additional language similar to what we have here: https://github.com/psharma1544/JupyterNotebooks/blob/master/LLMs/agentic_workflow.ipynb
- Potentially add cross checking where same daily value is retrieved from two distinct and unrelated sources giving higher confidence for the output.
